In [1]:
import sys
sys.path.append('../src/')
import pandas as pd
import kaggle
import auto_co2 as co2
from auto_co2.agg import CountryDataAggregator, ManufacturerDataAggregator, CarDataAggregator
import importlib
importlib.reload(co2)


<module 'auto_co2' from 'd:\\0 PERSO\\00 LUDO\\Google Drive\\0 LUDO\\Projets\\Cours_info\\05 Projet Dts\\github\\car-co2-prediction_pre-release\\notebooks\\../src\\auto_co2\\__init__.py'>

In [2]:
df = pd.read_csv('../data/sample/co2_raw_sample.csv')
df = co2.data.dataviz_preprocess(df)

Incomplete rows dropped:256


In [3]:
co2.styles.display_describe(df) 

,Colonne,count,mean,std,min,25%,50%,75%,max
0,ID,4744.000,47121348.062,15409019.889,458.000,46648102.000,51727626.000,54835141.000,61299368.000
1,MassRunningOrder,4744.000,1467.809,327.638,915.000,1235.000,1409.000,1624.000,3298.000
2,WltpTestMass,3864.000,1601.792,352.610,978.000,1352.000,1538.000,1772.250,3344.000
3,Co2EmissionsWltp,4744.000,116.085,51.586,0.000,112.000,126.000,141.000,373.000
4,BaseWheel,4744.000,2651.203,169.916,1873.000,2560.000,2640.000,2712.000,4490.000
5,AxleWidthSteering,4006.000,1548.930,60.334,1385.000,1513.000,1554.000,1580.000,2539.000
6,EngineCapacity,4295.000,1510.459,505.044,875.000,999.000,1496.000,1950.000,6749.000
7,EnginePower,4744.000,100.142,49.105,10.000,70.000,92.000,110.000,460.000
8,ElectricConsumption,675.000,172.570,37.894,45.000,150.000,165.000,180.000,373.000
9,InnovativeEmissionsReductionWltp,2166.000,1.520,0.511,0.500,1.000,1.600,1.900,4.300


<class 'pandas.io.formats.style.Styler'>


In [4]:
countries = CountryDataAggregator(df)
countries.display_sorted(n=5, by='ElectricRange', columns=['Country', 'GdpPerCapita', 'Count'], 
                         title="Top 5 des pays par capacité de propulsion électrique (moyenne kms)")
countries.countrywise_viz()

Columns: ['Country', 'GdpPerCapita', 'Count']
DataFrame columns: Index(['Country', 'Population', 'GdpPerCapita', 'Count', 'Make', 'FuelType',
       'FuelConsumption', 'EnginePower', 'Co2EmissionsWltp',
       'MassRunningOrder', 'BaseWheel', 'AxleWidthSteering', 'ElectricRange'],
      dtype='object')


,Country,GdpPerCapita,Count
24,SE,60375.043,105
17,LU,129396.306,19
20,NL,48569.467,144
6,DK,68349.344,91
0,AT,53493.772,116


<class 'pandas.io.formats.style.Styler'>


In [5]:
maker = ManufacturerDataAggregator(df)
maker.display_sorted(by='MarketShare(%)', n=8, title="Top 8 marques en Europe (2021)")

,,Count,MarketShare(%),FuelType,FuelConsumption,EnginePower,Co2EmissionsWltp,MassRunningOrder,BaseWheel,AxleWidthSteering,ElectricRange
Pool,Make,,,,,,,,,,
VW-SAIC,VOLKSWAGEN,538,11.340,PETROL,5.790,98.684,121.204,1499.015,2663.695,1536.193,27.390
RENAULT-NISSAN-MITSUBISHI,RENAULT,336,7.080,PETROL,5.214,72.655,101.967,1353.315,2626.345,1530.314,37.232
STELLANTIS,PEUGEOT,333,7.020,PETROL,4.907,84.943,111.859,1343.655,2619.850,1545.186,24.622
BMW,BMW,299,6.300,DIESEL,5.945,148.043,120.067,1777.030,2791.886,1592.374,34.692
MAZDA-SUBARU-SUZUKI-TOYOTA,TOYOTA,298,6.280,PETROL,4.441,85.218,109.218,1391.054,2622.886,1536.506,3.554
MERCEDES-BENZ,MERCEDES-BENZ,243,5.120,DIESEL,6.255,148.881,125.539,1852.568,2859.428,1612.948,34.926
VW-SAIC,SKODA,235,4.950,PETROL,5.484,100.409,124.098,1426.315,2649.715,1528.044,19.881
STELLANTIS,CITROEN,223,4.700,PETROL,5.009,79.758,118.525,1281.646,2619.332,1529.185,15.233


In [6]:
maker.plot_popular_fueltype()

In [7]:
maker.plot_mass_engine_scatter()

In [8]:
import plotly.express as px

# Get unique values
unique_df = df.drop_duplicates(subset=['CommercialName'])

# Create a scatter plot
fig = px.scatter(
    unique_df,
    x="WltpTestMass",
    y="EnginePower",
    color="Pool",
    hover_name="CommercialName", 
    labels={
        "MassRunningOrder": "Masse du véhicule",
        "EnginePower": "Puissance du moteur",
    },
    title="Comparaison de la masse et de la puissance des véhicules (KW)",
)
fig.show()

In [9]:
cars = CarDataAggregator(df)
cars.display_sorted(by='BaseWheel', ascending=True, n=10)
cars.specs(['TWINGO'])
#cars.specs('TWINGO')

Make,TWINGO
Pool,RENAULT-NISSAN-MITSUBISHI
FuelConsumption,5.292101
EnginePower,40.366667
Co2EmissionsWltp,58.766667
MassRunningOrder,1102.466667
BaseWheel,2492.000000
AxleWidthSteering,1467.978957
ElectricRange,81.333333
CarCount,30


In [10]:
cars.specs(['TWINGO', '208', 'PASSAT', 'C5', 'YARIS'])

Make,TWINGO,208,PASSAT,PASSAT,C5,YARIS
Pool,RENAULT-NISSAN-MITSUBISHI,STELLANTIS,VW-SAIC,VW-SAIC,STELLANTIS,MAZDA-SUBARU-SUZUKI-TOYOTA
FuelConsumption,5.292101,5.034991,5.487723,5.000000,5.458152,4.242269
EnginePower,40.366667,68.417582,113.666667,110.000000,96.000000,71.075000
Co2EmissionsWltp,58.766667,91.175824,112.948718,139.500000,138.000000,98.000000
MassRunningOrder,1102.466667,1223.527473,1609.512821,1437.000000,1574.000000,1188.225000
BaseWheel,2492.000000,2540.000000,2786.384615,2786.000000,2730.000000,2558.750000
AxleWidthSteering,1467.978957,1509.140742,1568.199732,1578.000000,1601.000000,1523.750000
ElectricRange,81.333333,52.175824,14.743590,0.000000,0.000000,0.000000
CarCount,30,91,41,41,1,40


In [11]:
cars.display_sorted(by=('Co2EmissionsWltp'), n=10, ascending=False, title='Top 10 des voitures les plus polluantes')

In [12]:
cars.display_sorted(by='Co2EmissionsWltp', ascending=True, n=10, title='Top 5 des voitures les moins polluantes')

In [13]:
cars.display_sorted(by=('FuelConsumption'), n=10, ascending=False, title='Top 10 des voitures les plus gourmandes')

,,Pool,FuelConsumption,EnginePower,Co2EmissionsWltp,MassRunningOrder,BaseWheel,AxleWidthSteering,ElectricRange,CarCount
Make,CommercialName,,,,,,,,,
AUDI,Q8 60 TFSI E,VW-SAIC,24.950,250.000,65.500,2566.000,2996.000,1681.500,45.000,2
MERCEDES-BENZ,AMG G 63,MERCEDES-BENZ,16.400,430.000,373.000,2560.000,2890.000,1657.000,0.000,2
ROLLS-ROYCE,GHOST,BMW,15.700,420.000,357.000,2565.000,3295.000,1674.000,0.000,1
NISSAN,NISSAN GT-R,RENAULT-NISSAN-MITSUBISHI,13.900,419.000,316.000,1821.000,2780.000,1590.000,0.000,1
BMW,X6 M COMPETITION,BMW,13.300,460.000,302.000,2395.000,2972.000,1688.000,0.000,1
MERCEDES-BENZ,AMG GLS 63 4MATIC+,MERCEDES-BENZ,12.800,450.000,290.000,2630.000,3135.000,1703.000,0.000,1
PORSCHE,PANAMERA GTS,VW-SAIC,12.600,353.000,286.000,2095.000,2950.000,1639.000,0.000,1
BMW,X7 M50I,BMW,12.450,390.000,284.500,2565.000,3105.000,1682.000,0.000,2
LAND ROVER,DEFENDER,TATA-MOTORS,12.200,221.000,85.000,2669.000,3022.000,1683.000,52.000,1


In [14]:
cars.display_sorted(by='Co2EmissionsWltp', ascending=True, n=10, title='Top 5 des voitures les moins polluantes')

In [15]:
cars.specs(['C3'])

Make,C3,C3
Pool,STELLANTIS,STELLANTIS
FuelConsumption,4.982602,5.458152
EnginePower,69.282353,75.000000
Co2EmissionsWltp,120.882353,110.000000
MassRunningOrder,1123.823529,1161.000000
BaseWheel,2540.541176,2539.000000
AxleWidthSteering,1511.536437,1483.000000
ElectricRange,0.000000,0.000000
CarCount,87,87


In [16]:
cars.plot_top_vehicles()

In [17]:
cars.display(n=5)

In [18]:
print(df[['FuelConsumption', 'EnginePower', 'Co2EmissionsWltp', 'MassRunningOrder', 'BaseWheel', 'AxleWidthSteering', 'ElectricRange']].isna().sum())

FuelConsumption      0
EnginePower          0
Co2EmissionsWltp     0
MassRunningOrder     0
BaseWheel            0
AxleWidthSteering    0
ElectricRange        0
dtype: int64


In [19]:
countries = CountryDataAggregator(df)
countries.display_sorted(n=5, by='ElectricRange', columns=['Country', 'GdpPerCapita', 'Count'], title="Top 5 des pays par capacité de propulsion électrique (moyenne kms)")

Columns: ['Country', 'GdpPerCapita', 'Count']
DataFrame columns: Index(['Country', 'Population', 'GdpPerCapita', 'Count', 'Make', 'FuelType',
       'FuelConsumption', 'EnginePower', 'Co2EmissionsWltp',
       'MassRunningOrder', 'BaseWheel', 'AxleWidthSteering', 'ElectricRange'],
      dtype='object')


,Country,GdpPerCapita,Count
24,SE,60375.043,105
17,LU,129396.306,19
20,NL,48569.467,144
6,DK,68349.344,91
0,AT,53493.772,116


In [20]:
countries.display_sorted(n=5, by='Co2EmissionsWltp', columns=['Country', 'GdpPerCapita', 'Count', 'Co2EmissionsWltp'], title="Top 5 des pays ayant les véhicules neufs les plus polluants en 2021")

Columns: ['Country', 'GdpPerCapita', 'Count', 'Co2EmissionsWltp']
DataFrame columns: Index(['Country', 'Population', 'GdpPerCapita', 'Count', 'Make', 'FuelType',
       'FuelConsumption', 'EnginePower', 'Co2EmissionsWltp',
       'MassRunningOrder', 'BaseWheel', 'AxleWidthSteering', 'ElectricRange'],
      dtype='object')


,Country,GdpPerCapita,Count,Co2EmissionsWltp
16,LT,24592.395,13,150.538
26,SK,21774.768,37,148.486
13,HU,18935.335,55,144.400
25,SI,30053.882,25,141.040
4,CZ,23152.189,113,139.920


In [21]:
co2.viz.plot_registrations_per_month(df)

In [22]:
co2.styles.display_info(df)

,Column,Non-Null Count,Dtype
0,ID,4744 non-null,int64
1,Country,4744 non-null,object
2,Pool,4744 non-null,object
3,ManufacturerName,4744 non-null,object
4,ManufNameOem,4744 non-null,object
5,Make,4744 non-null,object
6,CommercialName,4744 non-null,object
7,CategoryOf,4744 non-null,object
8,MassRunningOrder,4744 non-null,float64
9,WltpTestMass,3864 non-null,float64


,Info
0,"Index: 4744 entries, 0 to 4999"
1,Data columns (total 25 columns):
2,24 Month 4271 non-null float64
3,"dtypes: category(1), datetime64[ns](1), float64(12), int64(1), object(10)"
4,memory usage: 931.5+ KB


In [23]:
co2.styles.display_na(df)

,Colonne,Valeurs manquantes (%)
0,ElectricConsumption,85.77%
1,InnovativeEmissionsReductionWltp,54.34%
2,InnovativeTechnology,51.35%
3,WltpTestMass,18.55%
4,RegistrationDate,9.97%
5,Month,9.97%
6,EngineCapacity,9.46%
7,Country,0.00%
8,ID,0.00%
9,MassRunningOrder,0.00%


In [24]:
co2.styles.display_describe(df)

,Colonne,count,mean,std,min,25%,50%,75%,max
0,ID,4744.000000,47121348.061973,458.000000,46648102.000000,51727626.000000,54835141.000000,61299368.000000,15409019.888541
1,MassRunningOrder,4744.000000,1467.809444,915.000000,1235.000000,1409.000000,1624.000000,3298.000000,327.637628
2,WltpTestMass,3864.000000,1601.792443,978.000000,1352.000000,1538.000000,1772.250000,3344.000000,352.610054
3,Co2EmissionsWltp,4744.000000,116.084949,0.000000,112.000000,126.000000,141.000000,373.000000,51.586148
4,BaseWheel,4744.000000,2651.202993,1873.000000,2560.000000,2640.000000,2712.000000,4490.000000,169.916225
5,AxleWidthSteering,4744.000000,1548.929855,1385.000000,1521.000000,1548.929855,1574.000000,2539.000000,55.442124
6,EngineCapacity,4295.000000,1510.459371,875.000000,999.000000,1496.000000,1950.000000,6749.000000,505.043694
7,EnginePower,4744.000000,100.142074,10.000000,70.000000,92.000000,110.000000,460.000000,49.104801
8,ElectricConsumption,675.000000,172.570370,45.000000,150.000000,165.000000,180.000000,373.000000,37.894383
9,InnovativeEmissionsReductionWltp,2166.000000,1.519714,0.500000,1.000000,1.600000,1.900000,4.300000,0.511091


In [25]:
co2.viz.plot_fueltype_distribution(df)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4744 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ID                                4744 non-null   int64         
 1   Country                           4744 non-null   object        
 2   Pool                              4744 non-null   object        
 3   ManufacturerName                  4744 non-null   object        
 4   ManufNameOem                      4744 non-null   object        
 5   Make                              4744 non-null   object        
 6   CommercialName                    4744 non-null   object        
 7   CategoryOf                        4744 non-null   object        
 8   MassRunningOrder                  4744 non-null   float64       
 9   WltpTestMass                      3864 non-null   float64       
 10  Co2EmissionsWltp                  4744 non-null   flo

In [27]:
import plotly.graph_objects as go
co2.viz.plot_popular_fueltype(makers,save=False)


NameError: name 'makers' is not defined